Média de passos com política aleatória: 50.61
Média de passos com política ε-gulosa: 51.00


In [47]:
import numpy as np
import random

class GridworldEpsilonGreedy:
    def __init__(self, size=8, discount_factor=0.99, theta=0.01, epsilon=0.1):
        self.size = size
        self.discount_factor = discount_factor
        self.theta = theta
        self.epsilon = epsilon
        self.state_values = np.zeros((size, size))
        self.policy = np.ones((size, size, 4)) / 4
        self.mountains = [(2, 2), (3, 3), (4, 4)]
        self.quicksand = [(5, 2), (6, 3)]  #
        self.end = (7, 7)

    def is_terminal(self, state):
        return state == self.end

    def get_next_state_and_reward(self, state, action):
        if self.is_terminal(state):
            return state, 0
        if state in self.quicksand:
            return state, -100

        next_state = {
            'up': (max(state[0] - 1, 0), state[1]),
            'down': (min(state[0] + 1, self.size - 1), state[1]),
            'left': (state[0], max(state[1] - 1, 0)),
            'right': (state[0], min(state[1] + 1, self.size - 1))
        }.get(action, state)

        if next_state in self.mountains or next_state in self.quicksand:
            next_state = state

        reward = -100 if next_state in self.quicksand else -1
        return next_state, reward

    def evaluate_policy(self):
        while True:
            delta = 0
            for i in range(self.size):
                for j in range(self.size):
                    if self.is_terminal((i, j)):
                        continue
                    v_old = self.state_values[i, j]
                    v_new = 0
                    for action_prob, action in zip(self.policy[i, j], ["up", "down", "left", "right"]):
                        next_state, reward = self.get_next_state_and_reward((i, j), action)
                        v_new += action_prob * (reward + self.discount_factor * self.state_values[next_state])
                    self.state_values[i, j] = v_new
                    delta = max(delta, abs(v_old - v_new))
            if delta < self.theta:
                break

    def improve_policy(self):
        policy_stable = True
        for i in range(self.size):
            for j in range(self.size):
                if self.is_terminal((i, j)):
                    continue
                best_action_value = float('-inf')
                best_action = None
                for action_index, action in enumerate(["up", "down", "left", "right"]):
                    next_state, reward = self.get_next_state_and_reward((i, j), action)
                    action_value = reward + self.discount_factor * self.state_values[next_state]
                    if action_value > best_action_value:
                        best_action_value = action_value
                        best_action = action_index
                for action_index in range(4):
                    if action_index == best_action:
                        self.policy[i, j, action_index] = 1 - self.epsilon + (self.epsilon / 4)
                    else:
                        self.policy[i, j, action_index] = self.epsilon / 4
                if np.argmax(self.policy[i, j]) != best_action:
                    policy_stable = False
        return policy_stable

    def print_route(self, route):
        print("Rota criada:")
        for r in range(self.size):
            row = ""
            for c in range(self.size):
                if (r, c) in route:
                    row += " . "
                elif (r, c) in self.mountains:
                    row += " M "
                elif (r, c) in self.quicksand:
                    row += " Q "
                elif (r, c) == self.end:
                    row += " E "
                else:
                    row += " - "
            print(row)
        print()

    def simulate_optimal_policy(self):
        state = (0, 0)
        route = [state]
        steps = 0
        while not self.is_terminal(state) and state not in self.quicksand:
            action = ['up', 'down', 'left', 'right'][np.argmax(self.policy[state])]
            next_state, _ = self.get_next_state_and_reward(state, action)
            route.append(next_state)
            steps += 1
            if next_state == state or next_state in self.quicksand:
                break
            state = next_state
        return route, steps

    def print_values(self):
        print("Grid com valores esperados:")
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) in self.mountains:
                    print(" M ", end="")
                elif (i, j) in self.quicksand:
                    print(" Q ", end="")
                elif (i, j) == self.end:
                    print(" E ", end="")
                else:
                    print(f"{self.state_values[i, j]:.2f}", end=" ")
            print()
        print()

gridworld = GridworldEpsilonGreedy()
policy_stable = False
while not policy_stable:
    gridworld.evaluate_policy()
    policy_stable = gridworld.improve_policy()

gridworld.print_values()
route, steps = gridworld.simulate_optimal_policy()
gridworld.print_route(route)
print(f"Número de passos até o final: {steps}")


Grid com valores esperados:
-85.27 -84.82 -83.86 -82.43 -80.92 -79.53 -78.48 -77.92 
-85.13 -84.71 -83.68 -81.81 -80.02 -78.36 -77.13 -76.46 
-84.82 -84.58  M -80.39 -78.17 -75.90 -74.27 -73.39 
-84.12 -83.60 -82.43  M -75.49 -71.82 -69.63 -68.37 
-83.32 -82.58 -80.56 -75.88  M -65.14 -62.83 -60.81 
-82.58 -82.15  Q -70.22 -63.36 -59.35 -54.23 -49.65 
-81.56 -80.56 -78.50  Q -59.04 -53.02 -43.24 -31.88 
-80.80 -79.26 -75.57 -67.97 -59.08 -48.57 -31.52  E 

Rota criada:
 .  .  .  .  .  .  -  - 
 -  -  -  -  -  .  -  - 
 -  -  M  -  -  .  -  - 
 -  -  -  M  -  .  -  - 
 -  -  -  -  M  .  -  - 
 -  -  Q  -  -  .  -  - 
 -  -  -  Q  -  .  .  - 
 -  -  -  -  -  -  .  . 

Número de passos até o final: 14


In [49]:
import numpy as np

class GridworldPolicyIteration:
    def __init__(self, size=8, discount_factor=0.99, theta=0.01):
        self.size = size
        self.discount_factor = discount_factor
        self.theta = theta
        self.values = np.zeros((size, size))
        self.policy = np.zeros((size, size, 4)) + 0.25
        self.mountains = [(7, 2), (1, 3), (4, 4)]
        self.quicksand = [(5, 2), (6, 3)]
        self.end = (7, 7)

    def is_terminal(self, state):
        return state == self.end

    def get_next_state_and_reward(self, state, action):
        if self.is_terminal(state):
            return state, 0
        if state in self.quicksand:
            return state, -100

        r, c = state
        if action == 'up':
            next_r, next_c = max(r - 1, 0), c
        elif action == 'down':
            next_r, next_c = min(r + 1, self.size - 1), c
        elif action == 'left':
            next_r, next_c = r, max(c - 1, 0)
        elif action == 'right':
            next_r, next_c = r, min(c + 1, self.size - 1)

        if (next_r, next_c) in self.mountains or (next_r, next_c) in self.quicksand:
            next_r, next_c = r, c

        reward = -100 if (next_r, next_c) in self.quicksand else -1

        return (next_r, next_c), reward

    def policy_evaluation(self):
        while True:
            delta = 0
            for r in range(self.size):
                for c in range(self.size):
                    if self.is_terminal((r, c)):
                        continue
                    v_old = self.values[r, c]
                    v_new = 0
                    for action, action_prob in zip(['up', 'down', 'left', 'right'], self.policy[r, c]):
                        (next_r, next_c), reward = self.get_next_state_and_reward((r, c), action)
                        v_new += action_prob * (reward + self.discount_factor * self.values[next_r, next_c])
                    self.values[r, c] = v_new
                    delta = max(delta, abs(v_old - v_new))
            if delta < self.theta:
                break

    def policy_improvement(self):
        policy_stable = True
        for r in range(self.size):
            for c in range(self.size):
                if self.is_terminal((r, c)):
                    continue
                chosen_action = np.argmax(self.policy[r, c])
                action_values = []
                for action in ['up', 'down', 'left', 'right']:
                    (next_r, next_c), reward = self.get_next_state_and_reward((r, c), action)
                    action_values.append(reward + self.discount_factor * self.values[next_r, next_c])
                best_action = np.argmax(action_values)
                if chosen_action != best_action:
                    policy_stable = False
                self.policy[r, c] = np.eye(4)[best_action]
        return policy_stable

    def print_values(self):
        print("Grid com valores esperados:")
        for r in range(self.size):
            for c in range(self.size):
                if (r, c) in self.mountains:
                    print(" M ", end=" ")
                elif (r, c) in self.quicksand:
                    print(" Q ", end=" ")
                elif (r, c) == self.end:
                    print(" E ", end=" ")
                else:
                    print(f"{self.values[r, c]:5.2f}", end=" ")
            print()
        print()

    def simulate_optimal_policy(self):
        state = (0, 0)
        route = [state]
        steps = 0
        while not self.is_terminal(state) and state not in self.quicksand:
            action = ['up', 'down', 'left', 'right'][np.argmax(self.policy[state])]
            next_state, _ = self.get_next_state_and_reward(state, action)
            route.append(next_state)
            steps += 1
            if next_state == state or self.is_terminal(next_state):
                break
            state = next_state
        return route, steps

    def print_route(self, route):
        print("Rota criada:")
        for r in range(self.size):
            row = ""
            for c in range(self.size):
                if (r, c) in route:
                    row += " . "
                elif (r, c) in self.mountains:
                    row += " M "
                elif (r, c) in self.quicksand:
                    row += " Q "
                elif (r, c) == self.end:
                    row += " E "
                else:
                    row += " - "
            print(row)
        print()

gridworld = GridworldPolicyIteration()
policy_stable = False
while not policy_stable:
    gridworld.policy_evaluation()
    policy_stable = gridworld.policy_improvement()

gridworld.print_values()
route, steps = gridworld.simulate_optimal_policy()
gridworld.print_route(route)
print(f"Número de passos até o final: {steps}")

Grid com valores esperados:
-13.13 -12.25 -11.36 -10.47 -9.56 -8.65 -7.73 -6.79 
-12.25 -11.36 -10.47  M  -8.65 -7.73 -6.79 -5.85 
-11.36 -10.47 -9.56 -8.65 -7.73 -6.79 -5.85 -4.90 
-10.47 -9.56 -8.65 -7.73 -6.79 -5.85 -4.90 -3.94 
-9.56 -8.65 -7.73 -6.79  M  -4.90 -3.94 -2.97 
-10.47 -9.56  Q  -5.85 -4.90 -3.94 -2.97 -1.99 
-11.36 -10.47 -11.36  Q  -3.94 -2.97 -1.99 -1.00 
-12.25 -11.36  M  -3.94 -2.97 -1.99 -1.00  E  

Rota criada:
 .  -  -  -  -  -  -  - 
 .  -  -  M  -  -  -  - 
 .  -  -  -  -  -  -  - 
 .  -  -  -  -  -  -  - 
 .  .  .  .  M  -  -  - 
 -  -  Q  .  .  -  -  - 
 -  -  -  Q  .  -  -  - 
 -  -  M  -  .  .  .  . 

Número de passos até o final: 14


AttributeError: 'GridworldPolicyIteration' object has no attribute 'print_values'